# Numerical primal semidefinite program solver for the two-box problem

## Context

In this notebook the optimal expected correct-guessing probability $\text{P}_\text{corr}^*$ for MD of the 2BP is numerically computed by using the *picos* package to solve complex semidefinite program (SDP). Specifically, we solve the following (primal) SDP:

$$
\begin{align}
    \text{P}_\text{corr}^* = \max_{M_A}\quad & \eta_B + \text{Tr}(M_A(\eta_A\rho_A - \eta_B\rho_B))\\
    \text{subject to } \quad & M_A \succcurlyeq 0 \\
    & \mathrm{I} - M_A \succcurlyeq 0\\
\end{align}
$$

## Outputs

- Numerically computed minimum expected error probability

In [1]:
import numpy as np
import picos

In [2]:
# simulation parameters

n = 3 # half-number of photons
N = 2*n # total number of photons
d = 2**N # hilbert space dimension
etaA = 1/2. # prior probability of box A
etaB = 1-etaA # prior probability of box B

In [3]:
# load full density matrices

rhoA = np.load("../full_density_matrices/rhoA_N{:d}_num.npy".format(N))
rhoB = np.load("../full_density_matrices/rhoB_N{:d}_num.npy".format(N))

# Semidefinite Programming Solver

In [4]:
# set picos constants

rA = picos.Constant(etaA*rhoA)
rB = picos.Constant(etaB*rhoB)
I = picos.Constant(np.eye(d))

In [5]:
# sdp statement

P = picos.Problem()
MA = picos.HermitianVariable("MA", rA.shape)
P.set_objective("max", etaB + picos.trace(MA*(rA-rB)))
P.add_constraint(MA >> 0)
P.add_constraint(I - MA >> 0)
print(P)

Complex Semidefinite Program
  maximize 0.5 + tr(MA·([64×64] - [64×64]))
  over
    64×64 hermitian variable MA
  subject to
    MA ≽ 0
    [64×64] - MA ≽ 0


In [6]:
# solve

P.solve(solver = "cvxopt")
Perr = 1-P.value

In [7]:
# print and save data

print("\nOptimal expected failure probability:", Perr)
#print("Optimal X:", X.value, sep="\n")

# save optimal error probability
np.savetxt("data/perr_md_primal2bp_num_N{:d}.txt".format(N), [Perr])

# save optimal measurement
    
MAval = np.matrix(P.get_valued_variable("MA"))
MBval = np.matrix(I) - MAval

np.save("data/MA_md_2bp_num_N{:d}.npy".format(N), MAval)
np.save("data/MB_md_2bp_num_N{:d}.npy".format(N), MBval)


Optimal expected failure probability: 0.14174682600311184
